In [1]:
import tensorflow as tf 
from keras.datasets import fashion_mnist as fm

(X_train, y_train), (X_test, y_test) = fm.load_data()   
X_train, y_train = X_train[:55000], y_train[:55000]  # Use only the first 60,000 samples
X_valid,y_valid = X_train[-5000:], y_train[-5000:]

2025-07-09 02:48:57.665433: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

tf.random.set_seed(42)
from keras.layers import Normalization
norm_layer = Normalization(input_shape=(X_train.shape[1:]))

model = Sequential([
    norm_layer,
    Dense(50, activation='relu'),
    Dense(50, activation='relu'),
    Dense(50, activation='relu'),
    Dense(1)
])
o = Adam(learning_rate=1e-3)
model.compile(optimizer=o,loss="mse",metrics=["RootMeanSquaredError"])

norm_layer.adapt(X_train)

history = model.fit(X_train,y_train,epochs = 20, validation_data=(X_valid,y_valid))

mse_test, rmse_test = model.evaluate(X_test, y_test)

X_new = X_test[:3]
ypred = model.predict(X_new)

/opt/anaconda3/envs/deepL/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - RootMeanSquaredError: 3.0262 - loss: 9.2093 - val_RootMeanSquaredError: 2.8645 - val_loss: 8.2055
Epoch 2/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - RootMeanSquaredError: 2.8834 - loss: 8.3140 - val_RootMeanSquaredError: 2.8635 - val_loss: 8.1997
Epoch 3/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - RootMeanSquaredError: 2.8787 - loss: 8.2872 - val_RootMeanSquaredError: 2.8638 - val_loss: 8.2015
Epoch 4/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - RootMeanSquaredError: 2.8770 - loss: 8.2774 - val_RootMeanSquaredError: 2.8616 - val_loss: 8.1887
Epoch 5/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - RootMeanSquaredError: 2.8760 - loss: 8.2713 - val_RootMeanSquaredError: 2.8606 - val_loss: 8.1830
Epoch 6/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - RootMeanSquaredError: 2.8752 - loss: 8.2669 - val_RootMeanSquaredError: 2.8589 - val_loss: 8.1733
Epoch 7/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - RootMeanSquaredErr

In [3]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 28, 28)         │            57 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 28, 50)         │         1,450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 28, 50)         │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 28, 50)         │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 28, 1)          │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,862 (77.59 KB)

 Trainable params: 6,601 (25.79 KB)

 Non-trainable params: 57 (232.00 B)

 Optimizer params: 13,204 (51.58 KB)

In [ ]:
# Complex models using functional API
from keras.datasets import california_housing
from keras.models import Model
from keras.layers import Input, Dense, Normalization, Concatenate

(X_train, y_train), (X_test, y_test) = california_housing.load_data()
X_train, y_train = X_train[:14000], y_train[:14000]
X_valid, y_valid = X_train[14000:], y_train[14000:]

input_  = Input(shape=(X_train.shape[1:]))

norm_layer = Normalization()
normalized = norm_layer(input_)
norm_layer.adapt(X_train)

hidden1 = Dense(30, activation='relu')(normalized)
hidden2 = Dense(30, activation='relu')(hidden1)

concat = Concatenate()([input_, hidden2]) # we combine the two branches in order to have more knowledge for the last layer 

output = Dense(1)(concat)

model = Model(inputs =input_, outputs = output)


In [9]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 8)         │         17 │ input_layer_1[0]… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 30)        │        270 │ normalization_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 30)        │        930 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 38)        │          0 │ input_layer_1[0]… │
│ (Concatenate)       │                   │            │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │         39 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,256 (4.91 KB)

 Trainable params: 1,239 (4.84 KB)

 Non-trainable params: 17 (72.00 B)

In [ ]:
from keras.layers import Input, Normalization, Dense , Concatenate
from keras.models import Model
from keras.datasets import california_housing


(X_train, y_train), (X_test, y_test) = california_housing.load_data()
X_train, y_train = X_train[:14000], y_train[:14000]
X_valid, y_valid = X_train[14000:], y_train[14000:]

input_wide = Input(shape=[5], name="input_wide") # Features from 0 -> 4
input_deep = Input(shape =[2], name="input_deep") # Features from 2 -> 7

norm_layer_wide = Normalization()
norm_layer_deep = Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = Dense(30, activation="relu")(norm_deep)
hidden2 = Dense(30,activation="relu")(hidden1)

concat = Concatenate()([norm_wide,norm_deep])

output = Dense(1, name="output")(concat)
auxilary_output = Dense(1, name="aux_output")(hidden2) # Added ana auxilary output for he deep branch

model = Model(inputs=[input_wide, input_deep], outputs = [output,auxilary_output])

In [ ]:
X_train.shape
type(X_train_wide)
type(X_train_deep)  

numpy.ndarray

In [ ]:
from keras.optimizers import Adam
import tensorflow as tf 
tf.keras.backend.clear_session()

model.compile(loss = ("mse","mse"), loss_weights=(0.9, 0.1), metrics = ["RootMeanSquaredError"], optimizer = Adam(learning_rate=1e-3))

X_train_wide, X_train_deep = X_train[:, :5], X_train[:, -2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, -2:]

X_test_wide, X_test_deep = X_test[:, :5], X_test[:, -2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit(
    {"input_wide": X_train_wide, "input_deep": X_train_deep},
    {"output": y_train, "aux_output": y_train},
    epochs=20,
    validation_data=(
        {"input_wide": X_valid_wide, "input_deep": X_valid_deep},
        {"output": y_valid, "aux_output": y_valid}
    )
)

# Evaluate
eval_results = model.evaluate(
    {"input_wide": X_test_wide, "input_deep": X_test_deep},
    {"output": y_test, "aux_output": y_test},
    return_dict=True
)

weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse, rmse = eval_results 

y_pred_main, y_pred_aux = model.predict({"input_wide":X_new_wide, "input_deep": X_new_deep } )



Epoch 1/20


ValueError: Expected an instance of tuple, got <tf.Tensor 'functional_2_1/dense_11_1/Add:0' shape=(None, 1) dtype=float32>.